In [2]:
!pip install split_folders

In [3]:
import splitfolders

In [4]:
input_folder = r"C:\Users\Ishan's MSi\Desktop\Dataset\garbage_classification"
output = r"C:\Users\Ishan's MSi\Desktop\Dataset\process_data"
splitfolders.ratio(input_folder, output, seed=42, ratio=(.6, .2, .2))

Copying files: 15515 files [00:56, 274.88 files/s]


In [5]:
help(splitfolders.ratio)

Help on function ratio in module splitfolders.split:

ratio(input, output='output', seed=1337, ratio=(0.8, 0.1, 0.1), group_prefix=None, move=False)



#Training Model

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [8]:
def create_cnn_model(input_shape=(224, 224, 3), num_classes=12):
    model = Sequential()
    
    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Convolutional Layer 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten the layers
    model.add(Flatten())
    
    # Fully connected layer
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer with softmax activation
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [9]:
# Save the trained model to a file
def save_model(model, filepath='garbage_classification_model.h5'):
    model.save(filepath)
    print(f"Model saved to {filepath}")

In [10]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define image size, batch size, epochs, and learning rate
image_size = (150, 150)
batch_size = 32
epochs = 20
learning_rate = 0.001

# Define the directories for the dataset (train, validation, test)
train_dir = r"C:\Users\Ishan's MSi\Desktop\Dataset\process_data\train"
val_dir = r"C:\Users\Ishan's MSi\Desktop\Dataset\process_data\val"
test_dir = r"C:\Users\Ishan's MSi\Desktop\Dataset\process_data\test"


# Pre-processing and data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Creating data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Model architecture
model = Sequential()

# Convolutional and Pooling layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers and adding fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Helps to prevent overfitting
model.add(Dense(12, activation='softmax'))  # 12 classes for classification

# Compiling the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=val_generator.samples // batch_size
)

# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(val_generator)
print(f'Validation accuracy: {val_acc * 100:.2f}%')

# Test on test data
test_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Generate classification report and confusion matrix
Y_pred = model.predict(test_generator)
y_pred = Y_pred.argmax(axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


Found 9307 images belonging to 12 classes.
Found 3100 images belonging to 12 classes.


D:\Anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


D:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


290/290 ━━━━━━━━━━━━━━━━━━━━ 345s 1s/step - accuracy: 0.4045 - loss: 1.8932 - val_accuracy: 0.5785 - val_loss: 1.2407
Epoch 2/20
  1/290 ━━━━━━━━━━━━━━━━━━━━ 1:43 357ms/step - accuracy: 0.5625 - loss: 1.0868

D:\Anaconda\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5625 - loss: 1.0868 - val_accuracy: 0.5000 - val_loss: 1.7006
Epoch 3/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 223s 763ms/step - accuracy: 0.5823 - loss: 1.2859 - val_accuracy: 0.6517 - val_loss: 1.1113
Epoch 4/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.6250 - loss: 1.0864 - val_accuracy: 0.6786 - val_loss: 1.1818
Epoch 5/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 198s 674ms/step - accuracy: 0.6249 - loss: 1.1390 - val_accuracy: 0.6478 - val_loss: 1.0725
Epoch 6/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 558us/step - accuracy: 0.5938 - loss: 1.2816 - val_accuracy: 0.6071 - val_loss: 1.3252
Epoch 7/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 226s 775ms/step - accuracy: 0.6588 - loss: 1.0428 - val_accuracy: 0.7103 - val_loss: 0.9011
Epoch 8/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.7188 - loss: 0.8011 - val_accuracy: 0.7857 - val_loss: 0.5695
Epoch 9/20
290/290 ━━━━━━━━━━━━━━━━━━━━ 212s 720ms/step - accuracy: 0.6908 - loss: 0.9684 - val_a

In [27]:
pip install tk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tkinter import Tk, messagebox
from tkinter.filedialog import askopenfilename

# Use tkinter to open a file dialog to select the model file
Tk().withdraw()  # We don't want a full GUI, so keep the root window from appearing
model_path = askopenfilename(title="Select .h5 model file", filetypes=[("H5 files", "*.h5")])

if not model_path:
    messagebox.showerror("Error", "No file selected. Please select a valid .h5 file.")
    raise SystemExit("Exiting program due to missing file.")

try:
    # Load the trained model
    model = tf.keras.models.load_model(model_path)
    print(f"Model loaded successfully from {model_path}.")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    raise

# Assuming you have the test_generator and train_generator objects ready
# Below code assumes you have set them up correctly before running this part

# Prediction and Confusion Matrix
filenames = test_generator.filenames
nb_samples = len(test_generator)

y_prob = []
y_act = []

test_generator.reset()  # Resetting test generator to start from the first batch
for _ in range(nb_samples):
    X_test, y_test = next(test_generator)  # Get the next batch
    y_prob.append(model.predict(X_test))   # Predict the output
    y_act.append(y_test)                   # Collect the actual values

# Ensure numpy array conversion for predictions and actual values
y_prob = np.array(y_prob).reshape(-1, len(train_generator.class_indices))  # Flatten the list
y_act = np.array(y_act).reshape(-1, len(train_generator.class_indices))

# Convert predictions to class labels
predicted_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_prob]
actual_class = [list(train_generator.class_indices.keys())[i.argmax()] for i in y_act]

# Create a DataFrame for the actual vs predicted classes
out_df = pd.DataFrame(np.vstack([predicted_class, actual_class]).T, columns=['predicted_class', 'actual_class'])

# Confusion Matrix
confusion_matrix = pd.crosstab(out_df['actual_class'], out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, cmap="Blues", annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.show()

# Print test accuracy
accuracy = np.diag(confusion_matrix).sum() / confusion_matrix.sum().sum() * 100
print(f'Test Accuracy: {accuracy:.2f}%')



In [ ]:
# Load the trained model
model.save('CNN_garbage_classification.h5')

In [ ]:
# Importing necessary libraries
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


test_dir = r"D:\"

# ImageDataGenerator for test set
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

# Create the test data generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Get the number of test samples
nb_samples = test_generator.samples

# Initialize lists to store predictions and actual labels
y_prob = []
y_act = []

# Reset test generator before making predictions
test_generator.reset()

# Loop through test data and predict
for _ in range(nb_samples // test_generator.batch_size):
    X_test, y_test = next(test_generator)  # Get next batch of images
    y_prob.append(model.predict(X_test))  # Predict probabilities
    y_act.append(y_test)  # Append actual labels

# Convert lists to numpy arrays
y_prob = np.concatenate(y_prob)
y_act = np.concatenate(y_act)

# Get predicted and actual class indices
predicted_class = np.argmax(y_prob, axis=1)
actual_class = np.argmax(y_act, axis=1)

# Mapping class indices to class names
class_labels = list(test_generator.class_indices.keys())
predicted_class_names = [class_labels[i] for i in predicted_class]
actual_class_names = [class_labels[i] for i in actual_class]

# Create a DataFrame for the actual vs predicted classes
out_df = pd.DataFrame({
    'predicted_class': predicted_class_names,
    'actual_class': actual_class_names
})

# Confusion Matrix
confusion_mtx = pd.crosstab(out_df['actual_class'], out_df['predicted_class'], rownames=['Actual'], colnames=['Predicted'])

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, cmap="Blues", annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.show()

# Print test accuracy
accuracy = np.diag(confusion_mtx).sum() / confusion_mtx.sum().sum() * 100
print(f'Test Accuracy: {accuracy:.2f}%')

# Generate and print classification report
print('Classification Report')
print(classification_report(actual_class_names, predicted_class_names, target_names=class_labels))
